# Silicon calibration for a single crystal on ID11 nscope

Uses the older ImageD11 calibration fitting routines

Last updated 24/7/2024 by @jonwright

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( os.path.join( os.environ['HOME'],'git'), 'ImageD11_clean' )

In [ ]:
%matplotlib ipympl
import pylab as pl
import numpy as np
import ImageD11.sinograms.dataset
import ImageD11.transformer
import ImageD11.columnfile
import ImageD11.indexing
import ImageD11.grain
import ImageD11.nbGui.fit_geometry

In [ ]:
# 'Ag': 25.514, 'Sn': 29.2001, 'Nd': 43.5689, 'Gd': 50.2391, 'Hf': 65.3508, 'W' : 69.525, 'Pt': 78.3948, 'Pb': 88.0045
a_silicon = 5.43094
econst = 12.398423
energy = 43.5689

In [ ]:
# Load some peaks for your silicone
# colf = ImageD11.columnfile.columnfile('Silicon/Silicon_100ms/Silicon_100ms_peaks_2d.h5')
dset = ImageD11.sinograms.dataset.load("WAu_siliconAttrz25_dataset.h5")
colf = dset.get_cf_2d()

In [ ]:
# Remove any weak peaks / noise (average intensity > cutoff)
cutoff = 10
colf.filter(colf.sum_intensity / colf.Number_of_pixels > cutoff)

In [ ]:
f, a = pl.subplots(1,2,figsize=(12,6), constrained_layout=True)
a[0].plot(colf.fc,colf.sum_intensity/colf.Number_of_pixels,'.',alpha=0.2)
a[0].set(yscale='log',xlabel='pixel',ylabel='intensity')
a[1].plot(colf.fc,colf.sc,'.')
a[1].set(xlabel='pixel', ylabel='pixel');

In [ ]:
colf.writefile("si.flt")

In [ ]:
with open("si_start.par","w") as pars:
    pars.write(f"""cell__a {a_silicon}
cell__b {a_silicon}
cell__c {a_silicon}
cell_alpha 90.0
cell_beta 90.0
cell_gamma 90.0
cell_lattice_[P,A,B,C,I,F,R] 227
chi 0.0
distance 151000.0
fit_tolerance 0.05
o11 -1
o12 0
o21 0
o22 -1
omegasign 1.0
t_x 0
t_y 0
t_z 0
tilt_x 0.0
tilt_y 0.0
tilt_z 0.0
wavelength {econst/energy}
wedge 0.0
y_center 1050.4779244096353
y_size 75.0
z_center 1111.1724398511979
z_size 75.0""")

In [ ]:
ui = ImageD11.nbGui.fit_geometry.FitGeom( )
ui.loadfiltered( 'si.flt' )
ui.loadfileparameters( 'si_start.par' )
ui.fitGui()

In [ ]:
ui.valuewidgets['fit_tolerance'].value = 0.5
ui.fit(2,9)
ui.valuewidgets['fit_tolerance'].value = 0.05
ui.fit(3,21)
ui.fit(0,180)
ui.updateGui()

In [ ]:
ui.savegv('si.gve')
ui.saveparameters('si_powder.par')

In [ ]:
# Change log level to 1 to see what it did
idx = ImageD11.indexing.index( ui.colfile, npk_tol=[( ui.colfile.nrows//2, 0.05),], log_level=3)
idx.ubis

In [ ]:
idx.saveubis('si.ubi')

In [ ]:
!makemap.py -f si.flt -u si.ubi -U si.map -p si_powder.par -l cubic -s cubic -t 0.05 --omega_slop={dset.ostep/2}
!makemap.py -f si.flt -u si.map -U si.map -p si_powder.par -l cubic -s cubic -t 0.025 --omega_slop={dset.ostep/2}

In [ ]:
!refine_em.py si.flt.new si.map si_powder.par --omega_slop={dset.ostep/2} -x wedge -l cubic

In [ ]:
!cp 0.par si.par
!makemap.py -f si.flt -u si.map -U si.map -p si.par -l cubic -s cubic -t 0.05 --omega_slop={dset.ostep/2}
!makemap.py -f si.flt -u si.map -U si.map -p si.par -l cubic -s cubic -t 0.025 --omega_slop={dset.ostep/2}

In [ ]:
!refine_em.py si.flt.new si.map si.par --omega_slop={dset.ostep/2} -x wedge -l cubic

In [ ]:
!cp 0.par si.par
!cat si.par

In [ ]:
!makemap.py -f si.flt -u si.map -U si.map -p si.par -s cubic -t 0.025 --omega_slop={dset.ostep/2}

In [ ]:
!ubi2cellpars.py si.map

In [ ]:
g = ImageD11.grain.read_grain_file('si.map')[0]
v  = np.linalg.det(g.ubi)
a_avg = pow(v ,1/3)

deviatoric = g.eps_grain_matrix( [a_avg, a_avg, a_avg, 90, 90, 90] )
print('deviatoric strains, should be zero, so an estimate of precision:\n',deviatoric)

In [ ]:
wold = ui.parameterobj.get('wavelength')
wnew = wold*a_silicon/a_avg
print( 'Wavelength input',wold,'estimated from silicon',wnew)
print( 'Energy input',econst/wold,'estimated from silicon',econst/wnew)